In [1]:
from player.player import Player
from environments.simulator import Atari
import numpy as np
import datetime
from utils import HandleResults
import numba

GAME_ENV = 'BreakoutDeterministic-v4'
# GAME_ENV = 'SpaceInvaders-v4' # 758 frames
# GAME_ENV = 'Alien-v4' # 948 frames
# GAME_ENV = 'Amidar-v4' # 812 frames
# GAME_ENV = 'Venture-v4'
# GAME_ENV = 'Assault-v4' # 876 frames
# GAME_ENV = 'RoadRunner-v4' # 437 frames
# GAME_ENV = 'PongDeterministic-v4'
# GAME_ENV = 'Asterix-v4'
# GAME_ENV = 'MontezumaRevenge-v4'
# GAME_ENV = 'ChopperCommand-v4'
# OUT_FOLDER = './output/Punish_0_No_Reward_exploration/'
# OUT_FOLDER = './output/Punish_1_No_Reward_exploration/'
OUT_FOLDER = './output/Punish_1_Reward_exploration_2/'

results_handler = HandleResults(GAME_ENV, OUT_FOLDER)


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def run_episode(max_episode_length, episode, game_env, player, total_frames, evaluation=False):
    terminal_life_lost = game_env.reset()
    episode_reward = 0
    life_seq = 0
    frame_number = 0
    gif_frames = []
    while True:
        # Get state, make action, get next state (rewards, terminal, ...), record the experience, train if necessary
        current_state = game_env.get_current_state()
        action = player.take_action(current_state, episode, evaluation)
        processed_new_frame, reward, terminal, terminal_life_lost, original_frame = game_env.step(action)

        if frame_number >= max_episode_length:
            terminal = True
            terminal_life_lost = True

        # if evaluation:
        #     gif_frames.append(original_frame)

        if not evaluation:
            player.updates(total_frames, episode, action, processed_new_frame, reward, terminal_life_lost, life_seq)

        episode_reward += reward
        life_seq += 1

        if terminal_life_lost:
            life_seq = 0

        # game_env.env.render()
        total_frames += 1
        frame_number += 1

        if terminal:
            break

    return episode_reward, total_frames

In [3]:
load_folder=''
load_model=False

if load_folder is not '':
    player, game_env, max_episode_length, max_number_of_episodes, all_settings = \
        results_handler.load_settings(load_folder, load_model)
else:
    player, game_env, max_episode_length, max_number_of_episodes, all_settings = \
        results_handler.load_default_settings_constants(GAME_ENV)

results_handler.save_settings(all_settings, player)

The environment has the following 4 actions: ['NOOP', 'FIRE', 'RIGHT', 'LEFT']
Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 84, 84, 4)    0                                            
__________________________________________________________________________________________________
norm (Lambda)                   (None, 84, 84, 4)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 20, 20, 32)   8224        norm[0][0]                       
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 9, 9, 64)     3283

In [4]:
res_dict = {}

highest_reward = 0
total_frames = 0.0
prev_frames = 0.0
all_rewards = []
time = datetime.datetime.now()
prev_time = time
best_evaluation = 0
max_number_of_episodes = 1

for episode in range(max_number_of_episodes):
    episode_reward, total_frames = run_episode(max_episode_length, episode, game_env, player, total_frames)

    # all_rewards[episode] = episode_reward
    all_rewards.append(episode_reward)

    if episode_reward>highest_reward:
        highest_reward = episode_reward

    if episode % 10 == 0:
        # evaluation_reward, _ = run_episode(max_episode_length, episode, game_env, player, 0, evaluation=True)

        # if evaluation_reward > best_evaluation:
        #     best_evaluation = evaluation_reward
            # print('Best eval: ', str(best_evaluation))

        now = datetime.datetime.now()
        res_dict['time'] = str(now - time)
        res_dict['episode'] = episode
        res_dict['total_frames'] = total_frames
        res_dict['epsilon'] = format(player.epsilon, '.3f')
        res_dict['highest_reward'] = highest_reward
        # res_dict['best_eval'] = best_evaluation
        res_dict['mean_rewards'] = np.mean(all_rewards[-10:])
        res_dict['mean_loss'] = format(np.mean(player.losses[-10:]), '.5f')
        # res_dict['memory_vol'] = player.memory.count
        # res_dict['fps'] = (total_frames - prev_frames) / ((now - prev_time).total_seconds())
        res_dict['sparsity'] = np.mean(player.memory.sparsity_lengths[-10:])

        results_handler.save_res(res_dict)

        prev_time = now
        prev_frames = total_frames


C:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


{'time': '0:00:01.563906', 'episode': 0, 'total_frames': 199.0, 'epsilon': '1.000', 'highest_reward': 1.0, 'mean_rewards': 1.0, 'mean_loss': 'nan', 'sparsity': 31.333333333333332}


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
